In [1]:
import random
from math import log2, ceil
from decimal import Decimal
import benchmark_functions as bf
import scipy

class Chromosom:
    def __init__(self, precision, variables_count, start_, end_):
        self.precision = precision
        self.variables_count = variables_count
        self.start_ = start_
        self.end_ = end_
        self.chromosom_len = ceil(self.precision * log2(self.end_ - self.start_))
        self.chromosoms = self._generate_chromosom()
        self.decoded_chromosom = self._decode_chromosom()

    def _generate_chromosom(self) -> list:
      chromosoms = []
      for i in range(self.variables_count):
        chromosom = []
        for i in range(self.chromosom_len):
            chromosom.append(random.randint(0, 1))
        chromosoms.append(chromosom)
      return chromosoms

    def _decode_chromosom(self) -> list:
      decoded_chromosom = []
      for chromosom in self.chromosoms:
        decimal_number = sum(bit * (2 ** i) for i, bit in enumerate(reversed(chromosom)))
        decoded = self.start_ + decimal_number * (self.end_ - self.start_) / (2 ** self.chromosom_len - 1)
        decoded_chromosom.append(Decimal(decoded).quantize(Decimal(10) ** -self.precision))
      return decoded_chromosom
    
    def __str__(self):
        return f"Chromosoms: {self.chromosoms} | Value in Decimal: {self.decoded_chromosom}"


class Individual:
    def __init__(self, precision, variables_count, start_, end_):
        self.chromosom = Chromosom(precision, variables_count, start_, end_)
        self.variables_count = variables_count

    def __str__(self):
        return f"{self.chromosom}"


class Population:
    def __init__(self, variables_count, population_size, precision, start_, end_, func, optimum):
        self.variables_count = variables_count
        self.population_size = population_size
        self.func = func 
        self.individuals = [Individual(precision, variables_count, start_, end_) for _ in range(self.population_size)]
        self.optimum = 0 if optimum == "min" else 1
        self.cell = self.getCell()
        self.precision = precision

    def getCell(self) -> dict:
        """Oblicza wartości funkcji celu dla populacji."""
        cell_dict = {}
        X = self.getX()
        for x in X:
            x_tuple = tuple(x)  
            cell_value = self.func(x) 
            cell_dict[x_tuple] = cell_value  
        return cell_dict
        
    def getSortedCell(self) -> list[tuple[tuple[float], float]]:
        """Sortuje wartości funkcji celu."""
        return sorted(self.cell.items(), key=lambda item: item[1], reverse=self.optimum)

    def getBestBySelection(self, percentage: float,) -> list:
        """Zwraca najlepsze osobniki według funkcji celu."""
        size = int(percentage * self.population_size / 100)
        best_individuals = self.getSortedCell()
        return best_individuals[:size]
      
    # def getBestByTournament(self, k) -> list:
    #     """Zwraca najlepsze osobniki według funkcji celu."""
    #     if k > self.population_size:
    #         raise ValueError("k parameter can not be grater then population_size!")
    #     cell = self.cell
    #     best_individuals = []
    #     for i in range(k):
    #       for j in range(int(self.population_size/k)):
    #         individual = random.choice(list(cell.items()))
    #         best_individuals.append(individual)
    #         cell.pop(individual.index)
    #     return best_individuals
    
    def getBestByTournament(self, k) -> list:
      """Zwraca najlepsze osobniki według funkcji celu."""
      if k > self.population_size:
          raise ValueError("k parameter cannot be greater than population_size!")
      
      cell = self.cell.copy()
      best_individuals = []

      for _ in range(k):
          tournament = random.sample(list(cell.items()), int(self.population_size / k))  # Losujemy podzbiór
          if self.optimum == 1:
            best = max(tournament, key=lambda x: x[1])  # Wybieramy najlepszego osobnika (dla maksymalizacji)
          else:
            best = min(tournament, key=lambda x: x[1])  # Wybieramy najlepszego osobnika (dla minimalizacji)
          best_individuals.append(best)
          del cell[best[0]]  # Usuwamy wybranego osobnika

      return best_individuals

    def getBestByRulet(self) -> list:
        """Zwraca najlepsze osobniki według funkcji celu."""
        cell = self.cell.copy()
        best_individuals = []
        if self.optimum == 0:
          cell = {key: 1/value for key, value in cell.items()}
        total_fitness = sum(cell.values())
        if total_fitness == 0:
            raise ValueError("Total fitness is zero!")

        probabilities = {}
        distribution = {}
        distribution_value = 0

        for value in cell.values():
            probability = value / total_fitness
            probabilities[value] = probability
            distribution_value += probability
            distribution[value] = distribution_value

        if round(sum(probabilities.values())) != 1.0:
            raise ValueError("Sum of probabilities is not equal to 1!")
        last_item = round(list(distribution.items())[-1][1])
        if last_item != 1.0:
            raise ValueError("Last distribution value is not equal to 1!")
        num = random.random()
        for key, value in distribution.items():
            if num <= value:
                x = next(k for k, v in cell.items() if v == key)
                best_individuals.append((x, key))  # Zapisujemy klucz + wartość
                del cell[x]  # Usuwamy osobnika
                break
        return best_individuals
    
    #krzyżowanie jednopunktowe
    def single_point_crossover(self, parent1, parent2):
        """Krzyżowanie jednopunktowe dla chromosomów binarnych."""
        child1_chromosoms = []
        child2_chromosoms = []
    
        print("Parent 1 chromosoms:", parent1.chromosom.chromosoms)
        print("Parent 2 chromosoms:", parent2.chromosom.chromosoms)
    
        # Iterujemy po każdej zmiennej w chromosomie (bo może być ich kilka)
        for p1_chromo, p2_chromo in zip(parent1.chromosom.chromosoms, parent2.chromosom.chromosoms):
            end_of_range = len(p1_chromo)
            k = random.randint(1, end_of_range - 1)  # Punkt krzyżowania (nie może być 0)
            print(f"Crossover point: {k}")
    
            # Tworzymy nowe chromosomy dzieci
            new_p1 = p1_chromo[:k] + p2_chromo[k:]
            new_p2 = p2_chromo[:k] + p1_chromo[k:]
    
            child1_chromosoms.append(new_p1)
            child2_chromosoms.append(new_p2)
    
        # Tworzymy nowe osobniki
        child1 = Individual(parent1.chromosom.precision, parent1.variables_count, parent1.chromosom.start_, parent1.chromosom.end_)
        child2 = Individual(parent2.chromosom.precision, parent2.variables_count, parent2.chromosom.start_, parent2.chromosom.end_)
    
        # Podmieniamy chromosomy na nowe
        child1.chromosom.chromosoms = child1_chromosoms
        child2.chromosom.chromosoms = child2_chromosoms
    
        print("Child 1 chromosoms:", child1.chromosom.chromosoms)
        print("Child 2 chromosoms:", child2.chromosom.chromosoms)
    
        return child1, child2
    
    #krzyżowanie dwupunktowe
    def two_point_crossover(self, parent1, parent2, min_gap=1):
        """Krzyżowanie dwupunktowe dla chromosomów binarnych."""
        child1_chromosoms = []
        child2_chromosoms = []
    
        print("Parent 1 chromosoms:", parent1.chromosom.chromosoms)
        print("Parent 2 chromosoms:", parent2.chromosom.chromosoms)
    
        # Iterujemy po każdej zmiennej w chromosomie (bo może być ich kilka)
        for p1_chromo, p2_chromo in zip(parent1.chromosom.chromosoms, parent2.chromosom.chromosoms):
            end_of_range = len(p1_chromo)
            point1 = random.randint(1, end_of_range - 1)  # Pierwszy punkt krzyżowania (nie może być 0)
            while True:
                point2 = random.randint(1, end_of_range - 1)
                if point2 != point1 and abs(point2 - point1) >= min_gap:
                    break
            print(f"Crossover points: {point1} {point2}")
            
            lower = min(point1, point2)
            upper = max(point1, point2)
            print(lower, upper)
            # Tworzymy nowe chromosomy dzieci
            new_p1 = p1_chromo[:lower] + p2_chromo[lower:upper] + p1_chromo[upper:]
            new_p2 = p2_chromo[:lower] + p1_chromo[lower:upper] + p2_chromo[upper:]
            
            child1_chromosoms.append(new_p1)
            child2_chromosoms.append(new_p2)
            
        # Tworzymy nowe osobniki
        child1 = Individual(parent1.chromosom.precision, parent1.variables_count, parent1.chromosom.start_, parent1.chromosom.end_)
        child2 = Individual(parent2.chromosom.precision, parent2.variables_count, parent2.chromosom.start_, parent2.chromosom.end_)
    
        # Podmieniamy chromosomy na nowe
        child1.chromosom.chromosoms = child1_chromosoms
        child2.chromosom.chromosoms = child2_chromosoms
    
        print("Child 1 chromosoms:", child1.chromosom.chromosoms)
        print("Child 2 chromosoms:", child2.chromosom.chromosoms)
        
        return child1, child2
    
    #krzyżowanie jednorodne
    def uniform_crossover(self, parent1, parent2, cross_probability=1.0):
        """Krzyżowanie jednorodne dla chromosomów binarnych."""
        child1_chromosoms = []
        child2_chromosoms = []      
        
        print("Parent 1 chromosoms:", parent1.chromosom.chromosoms)
        print("Parent 2 chromosoms:", parent2.chromosom.chromosoms)
        
        for p1_chromo, p2_chromo in zip(parent1.chromosom.chromosoms, parent2.chromosom.chromosoms):
            child1_genes = []
            child2_genes = []
            
            for gene1, gene2 in zip(p1_chromo, p2_chromo):  
                gene_random_rate = random.uniform(0, 1)
                print("Gene random rate:", gene_random_rate)
    
                if gene_random_rate <= cross_probability:
                    # Zamieniamy geny
                    child1_genes.append(gene2)
                    child2_genes.append(gene1)
                else:
                    # Zostają takie same
                    child1_genes.append(gene1)
                    child2_genes.append(gene2)
            
            # Dodajemy nowo utworzone chromosomy do listy chromosomów dzieci
            child1_chromosoms.append(child1_genes)
            child2_chromosoms.append(child2_genes)
    
        # Tworzymy nowe osobniki
        child1 = Individual(parent1.chromosom.precision, parent1.variables_count, parent1.chromosom.start_, parent1.chromosom.end_)
        child2 = Individual(parent2.chromosom.precision, parent2.variables_count, parent2.chromosom.start_, parent2.chromosom.end_)
    
        # Podmieniamy chromosomy na nowe
        child1.chromosom.chromosoms = child1_chromosoms
        child2.chromosom.chromosoms = child2_chromosoms
    
        print("Child 1 chromosoms:", child1.chromosom.chromosoms)
        print("Child 2 chromosoms:", child2.chromosom.chromosoms)
        
        return child1, child2 
    
    def population_after_crossover(self, crossover_method_number, crossover_rate=1.0):
        """Wykonuje jednopunktowe krzyżowanie dla całej populacji."""
        new_population = []
        individuals = self.individuals[:]
        random.shuffle(individuals)
        
        for i in range(0, len(individuals) - 1, 2):
            parent1, parent2 = individuals[i], individuals[i + 1]
            
            if random.random() < crossover_rate:
                if crossover_method_number == 1:
                    child1, child2 = self.single_point_crossover(parent1, parent2)
                elif crossover_method_number == 2:
                    child1, child2 = self.two_point_crossover(parent1, parent2)
                elif crossover_method_number == 3:
                    child1, child2 = self.uniform_crossover(parent1, parent2, cross_probability=0.7)
                new_population.extend([child1, child2])
            else:
                new_population.extend([parent1, parent2])
        
        self.individuals = new_population
        
    def getX(self) -> list:
        """Zwraca listę fenotypów (wartości zmiennych)."""
        return [[float(x) for x in individual.chromosom.decoded_chromosom] for individual in self.individuals]

    def __str__(self):
        return "\n".join(str(individual) for individual in self.individuals)



In [2]:
c = Chromosom(10, 5, -10, 10)
chromosoms = c.chromosoms
decoded_chromosom = c.decoded_chromosom
print(c)


Chromosoms: [[0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0], [0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1], [1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0], [0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0], [0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0]] | Value in Decimal: [Decimal('-4.0313743130'), Decimal('-5.6188896144'), Decimal('1.0278209967'), Decimal('-8.8182531620'), Decimal('-5.0310708144')]


In [3]:
func = bf.Schwefel(n_dimensions=3)

population = Population(3, 10, 10, -10, 10, func, "min")

population.getBestByRulet()


[((9.9526068855, -5.3253875268, 5.3390217913), 0.0007954940611458965)]

In [5]:
population.population_after_crossover(3, crossover_rate=1)


Parent 1 chromosoms: [[0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1], [1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0], [1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1]]
Parent 2 chromosoms: [[0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1], [1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1], [0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0]]
Gene random rate: 0.006504558678395278
Gene random rate: 0.2952660780369114
Gene random rate: 0.5957788172880096
Gene random rate: 0.8886245772325654
Ge